In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
# Cargar los datos
datos = pd.read_csv('./assets/data_test.csv')

datos = datos[['fecha', 'FZN150D-6 (FZ-S)']]
datos = datos.rename(columns={"FZN150D-6 (FZ-S)": "ventas"})

datos['fecha'] = pd.to_datetime(datos['fecha'], format='%Y')

datos.insert(1, 'dia', (datos['fecha'].dt.to_period('D') + 1 - datos['fecha'].dt.to_period('D').min()).apply(lambda x: x.n))
datos.insert(1, 'mes', (datos['fecha'].dt.to_period('M') + 1 - datos['fecha'].dt.to_period('M').min()).apply(lambda x: x.n))

datos.head(10)

,fecha,mes,dia,ventas
0,2020-01-01,1,1,3
1,2020-01-01,1,1,0
2,2020-01-01,1,1,2
3,2020-01-01,1,1,2
4,2020-01-01,1,1,1


In [8]:
print(datos['fecha'])

0   2020-01-01
1   2020-01-01
2   2020-01-01
3   2020-01-01
4   2020-01-01
Name: fecha, dtype: datetime64[ns]


In [4]:
# Límites basados en cuartiles
Q1 = np.percentile(datos['ventas'], 25)
Q3 = np.percentile(datos['ventas'], 75)
IQR = Q3 - Q1
rango_valido = [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR]

# Límites basados en desviaciones estándar
# media = datos['ventas'].mean()
# desv_std = datos['ventas'].std()
# rango_valido = [media - 3 * desv_std, media + 3 * desv_std]

# Filtrar datos dentro del rango válido
datos_filtrados = datos[(datos['ventas'] >= rango_valido[0]) & (datos['ventas'] <= rango_valido[1])]
datos_filtrados.head(10)

,fecha,mes,dia,ventas
0,2020-01-02,1,1,3
1,2020-01-03,1,2,0
2,2020-01-04,1,3,2
3,2020-01-07,1,6,2
4,2020-01-08,1,7,1
5,2020-01-09,1,8,2
6,2020-01-10,1,9,2
8,2020-01-13,1,12,7
9,2020-01-14,1,13,0
10,2020-01-15,1,14,1


In [5]:
# datos = datos[('2023-01-01' <= datos['fecha'])]
# time = np.asarray(datos['fecha'], dtype='datetime64[s]')

In [6]:
# Límites basados en cuartiles
Q1 = np.percentile(datos['ventas'], 25)
Q3 = np.percentile(datos['ventas'], 75)
IQR = Q3 - Q1
rango_valido = [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR]

In [7]:
# Límites basados en desviaciones estándar
media = datos['ventas'].mean()
desv_std = datos['ventas'].std()
rango_valido = [media - 3 * desv_std, media + 3 * desv_std]

In [8]:
# Filtrar datos dentro del rango válido
datos_filtrados = datos[(datos['ventas'] >= rango_valido[0]) & (datos['ventas'] <= rango_valido[1])]
datos_filtrados.head(10)

,fecha,mes,dia,ventas
0,2020-01-02,1,1,3
1,2020-01-03,1,2,0
2,2020-01-04,1,3,2
3,2020-01-07,1,6,2
4,2020-01-08,1,7,1
5,2020-01-09,1,8,2
6,2020-01-10,1,9,2
7,2020-01-11,1,10,16
8,2020-01-13,1,12,7
9,2020-01-14,1,13,0


In [9]:
# Preparar los datos
X = datos[['dia', 'mes']]
# X = datos['dia'].values.reshape(-1,1)
y = datos['ventas']

In [10]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)

In [11]:
# Crear características polinómicas
poli_features = PolynomialFeatures(degree=2)
X_poli = poli_features.fit_transform(X)

# Crear el modelo de regresión polinómica
modelo_poli = LinearRegression()

# Configurar la validación cruzada k-fold
k = 5  # Número de divisiones (folds)
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

# Evaluar el modelo utilizando validación cruzada
scores = cross_val_score(modelo_poli, X_poli, y, cv=kfold, scoring='r2')

# Imprimir los resultados
print(f"Puntajes de R^2 en validación cruzada: {scores}")
print(f"Promedio de R^2: {scores.mean():.3f}")

Puntajes de R^2 en validación cruzada: [0.11381414 0.06868068 0.06035519 0.06859389 0.06009264]
Promedio de R^2: 0.074


In [12]:
# Crear características polinómicas de grado 2 (cuadrático)
# grado_polinomio = 4
# poli_features = PolynomialFeatures(degree=grado_polinomio)
# X_train_poli = poli_features.fit_transform(X_train)
# X_test_poli = poli_features.transform(X_test)

In [13]:
# Crear y entrenar el modelo de regresión polinómica
# modelo_poli = LinearRegression()
# modelo_poli.fit(X_train_poli, y_train)

In [14]:
# Hacer predicciones en el conjunto de prueba
# y_pred = modelo_poli.predict(X_test_poli)

In [15]:
# Evaluar el rendimiento del modelo
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Error cuadrático medio (MSE): {mse}')
# print(f'Coeficiente de determinación (R^2): {r2}')

In [16]:
# Realizar predicciones
# mes_siguiente = 49
# ventas_predichas = modelo.predict([[mes_siguiente]])
# print(f"Las ventas predichas para el mes {mes_siguiente} son: {ventas_predichas[0]}")